# Klavis Filesystem MCP Sandbox API Example

This notebook demonstrates the core sandbox lifecycle endpoints for **Filesystem** (local_dev):
- **Acquire** a local_dev sandbox instance (includes filesystem, git, terminal, and more)
- **Get** sandbox details  
- **Initialize** by uploading files
- **Export** sandbox files as a tar.gz archive
- **Reset** to initial state
- **Release** the sandbox

## 1. Setup

In [ ]:
import os
import httpx
from dotenv import load_dotenv

load_dotenv()

BASE_URL = "https://api.klavis.ai"
KLAVIS_API_KEY = os.environ.get("KLAVIS_API_KEY")

headers = {"Authorization": f"Bearer {KLAVIS_API_KEY}"}

SERVER_NAME = "local_dev"

## 2. Acquire Local Dev Sandbox

`POST /sandbox/local_dev` - Acquire a local_dev sandbox instance (includes filesystem, git, terminal, and other MCP servers)

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}",
        headers=headers
    )

SANDBOX_ID = resp.json()["sandbox_id"]
print(resp.json())

## 3. Get Sandbox Details

`GET /sandbox` - Get sandbox info that you occupied

`GET /sandbox/local_dev/{sandbox_id}` - Get sandbox information

In [ ]:
# Get your sandbox
async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox",
        headers=headers
    )

print(resp.json())

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}",
        headers=headers
    )

print(resp.json())

## 4. Initialize Filesystem with Files

Uploading files is a **two-step** process via GCS:

1. `POST /sandbox/local_dev/{sandbox_id}/upload-url` — get a signed GCS upload URL
2. `PUT` the tar.gz archive directly to that signed URL
3. `POST /sandbox/local_dev/{sandbox_id}/initialize` — tell the server to extract it into the pod (no request body needed)

In [ ]:
import tarfile
import io

# Create example files to upload
example_files = {
    "hello.txt": "Hello from the Klavis Filesystem Sandbox!\nThis is a sample text file.",
    "notes.md": "# Project Notes\n\n## TODO\n- Set up development environment\n- Write unit tests\n- Deploy to staging",
    "config.json": '{\n  "app_name": "sandbox-demo",\n  "version": "1.0.0",\n  "debug": true\n}'
}

# Pack files into a tar.gz archive in memory
buf = io.BytesIO()
with tarfile.open(fileobj=buf, mode="w:gz") as tar:
    for name, content in example_files.items():
        data = content.encode("utf-8")
        info = tarfile.TarInfo(name=name)
        info.size = len(data)
        tar.addfile(info, io.BytesIO(data))
buf.seek(0)

print(f"Created tar.gz archive ({buf.getbuffer().nbytes} bytes) with {len(example_files)} files")

In [ ]:
# Step 1: Get a signed GCS upload URL
async with httpx.AsyncClient() as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/upload-url",
        headers=headers
    )

upload_info = resp.json()
upload_url = upload_info["upload_url"]
print(f"Got signed upload URL (expires in {upload_info['expires_in_minutes']} minutes)")

# Step 2: PUT the tar.gz archive directly to the signed GCS URL
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.put(
        upload_url,
        headers={"Content-Type": "application/gzip"},
        content=buf.getvalue()
    )

print(f"Upload to GCS: {resp.status_code}")

# Step 3: Tell the server to extract the archive into the pod
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/initialize",
        headers=headers
    )

print(resp.json())

## 5. Export Filesystem Sandbox

`GET /sandbox/local_dev/{sandbox_id}/dump` - Get a signed GCS download URL for all sandbox files as a gzip-compressed tar archive (tar.gz)

1. `GET /sandbox/local_dev/{sandbox_id}/dump` — get a signed GCS download URL
2. `GET` the signed URL to download the actual tar.gz archive

In [ ]:
# Step 1: Get a signed GCS download URL
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/dump",
        headers=headers
    )

dump_info = resp.json()
download_url = dump_info["download_url"]
print(f"Got signed download URL (expires in {dump_info['expires_in_minutes']} minutes)")

# Step 2: Download the tar.gz archive from the signed GCS URL
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.get(download_url)
    tar_gz_bytes = resp.content

# List contents of the downloaded archive
tar = tarfile.open(fileobj=io.BytesIO(tar_gz_bytes), mode="r:gz")
print("Files in sandbox:")
for member in tar.getmembers():
    print(f"  {member.name} ({member.size} bytes)")
tar.close()

## 6. Reset Filesystem Sandbox

`POST /sandbox/local_dev/{sandbox_id}/reset` - Clear all files, for re-use this sandbox

In [ ]:
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/reset",
        headers=headers
    )

print(resp.json())

## 7. Release Filesystem Sandbox

`DELETE /sandbox/local_dev/{sandbox_id}` - Release back to idle pool

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.delete(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}",
        headers=headers
    )

print(resp.json())